# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import google_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Create vacation dataframe
vacation_df = pd.read_csv('../output_data/cities.csv')

vacation_df.count()
vacation_df.head()

,City,Lat,Lng,Max temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Ushuaia,-54.80,-68.30,28.400,92,70,2.60,AR,2020-07-16 11:27:46
1,Bredasdorp,-34.53,20.04,64.400,48,0,2.60,ZA,2020-07-16 11:27:46
2,Redmond,47.67,-122.12,62.006,89,10,0.31,US,2020-07-16 11:23:29
3,Minusinsk,53.71,91.69,71.600,40,79,1.00,RU,2020-07-16 11:27:46
4,Busselton,-33.65,115.33,55.994,75,100,3.95,AU,2020-07-16 11:27:46


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
locations = vacation_df[["Lat", "Lng"]]

# Fill NaN values and convert to float
humidity = vacation_df["Humidity"]

In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* A max temperature lower than 80 degrees but higher than 70.
* Wind speed less than 10 mph
* Zero cloudiness.
* Drop any rows will null values.

In [5]:
#Narrow down the cities to fit weather conditions

clean_vacation_df = vacation_df.loc[(vacation_df["Max temp"] <80) & 
                                    (vacation_df["Max temp"] >70) &
                                    (vacation_df["Wind Speed"] <10)&
                                    (vacation_df["Cloudiness"]==0), :]

#drop null values and reset index
clean_vacation_df = clean_vacation_df.dropna(how='any')
clean_vacation_df.reset_index(inplace=True)
del clean_vacation_df['index']
clean_vacation_df.head()


,City,Lat,Lng,Max temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Mossamedes,-15.20,12.15,71.600,68,0,5.10,AO,2020-07-16 11:27:52
1,Cape Town,-33.93,18.42,71.006,45,0,1.00,ZA,2020-07-16 11:27:54
2,Cabo San Lucas,22.89,-109.91,78.008,90,0,4.27,MX,2020-07-16 11:27:59
3,Caucaia,-3.74,-38.65,78.998,69,0,6.20,BR,2020-07-16 11:28:37


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# copy df to hotel_df
hotel_df=clean_vacation_df
hotel_df.head()

,City,Lat,Lng,Max temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Mossamedes,-15.20,12.15,71.600,68,0,5.10,AO,2020-07-16 11:27:52
1,Cape Town,-33.93,18.42,71.006,45,0,1.00,ZA,2020-07-16 11:27:54
2,Cabo San Lucas,22.89,-109.91,78.008,90,0,4.27,MX,2020-07-16 11:27:59
3,Caucaia,-3.74,-38.65,78.998,69,0,6.20,BR,2020-07-16 11:28:37


In [7]:
#define variable to store hotel name
hotel_list = []

#iterate through hotel_df dataframe
for i in range(len(hotel_df)):
    lat_coord = hotel_df.loc[i]['Lat']
    lng_coord = hotel_df.loc[i]['Lng']
    
# set up a parameters dictionary
    params = {
        "location": f"{lat_coord},{lng_coord}",
        "radius": 5000,
        "types" : "hotel",
        "key": google_key
        
    }
    
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # assemble url and make API request
    request = requests.get(base_url, params=params)
    response = request.json()
    
    try:
        hotel_list.append(response['results'][0]['name'])
        print(f"Closest hotel is {response['results'][0]['name']}.")
    except(KeyError, IndexError):
        print("Missing field/result.")
        hotel_list.append("")
        
# Add new column to store Hotel Name        
hotel_df["Hotel Name"] = hotel_list

hotel_df.head()

Closest hotel is Namibe.
Closest hotel is Cape Town.
Closest hotel is Cabo San Lucas.
Closest hotel is Fortaleza.


,City,Lat,Lng,Max temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Mossamedes,-15.20,12.15,71.600,68,0,5.10,AO,2020-07-16 11:27:52,Namibe
1,Cape Town,-33.93,18.42,71.006,45,0,1.00,ZA,2020-07-16 11:27:54,Cape Town
2,Cabo San Lucas,22.89,-109.91,78.008,90,0,4.27,MX,2020-07-16 11:27:59,Cabo San Lucas
3,Caucaia,-3.74,-38.65,78.998,69,0,6.20,BR,2020-07-16 11:28:37,Fortaleza


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))